# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession, functions as F, Window
import json

In [2]:
# Initialize Spark session
se = SparkSession.builder.appName('clickstream').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 18:08:57,310 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
# Step 1: Load the CSV into a DataFrame
clickstream_df = se.read.csv("hdfs:///data/clickstream.csv", sep='\t', header=True)

Spark SQL

In [4]:
# Step 2: Clean data using Spark SQL (remove rows where event_type contains 'error')
clickstream_df.createOrReplaceTempView("clickstream")

cleaned_data_sql = se.sql("""
    SELECT user_id, session_id, event_page, timestamp
    FROM clickstream
    WHERE event_type NOT LIKE '%error%'
    ORDER BY user_id, session_id, timestamp
""")

cleaned_data_sql.show(10)

+-------+----------+----------+----------+
|user_id|session_id|event_page| timestamp|
+-------+----------+----------+----------+
|      0|       874|      main|1696371064|
|      0|       874|      main|1696372696|
|      0|       874|      main|1696373564|
|      0|       874|    rabota|1696374894|
|      0|       874|    rabota|1696377393|
|      0|       874|    online|1696378229|
|      0|       874|    online|1696378928|
|      0|       879|      main|1696768416|
|      0|       879|    online|1696768738|
|      0|       879|   tariffs|1696768973|
+-------+----------+----------+----------+
only showing top 10 rows



Spark DF

In [5]:
# Step 3: Remove consecutive duplicates and create routes (Spark DF)
from pyspark.sql.types import ArrayType, StringType

def remove_consecutive_duplicates(pages):
    if not pages:
        return pages
    result = [pages[0]]
    for i in range(1, len(pages)):
        if pages[i] != pages[i-1]:
            result.append(pages[i])
    return result

In [7]:
remove_duplicates_udf = F.udf(remove_consecutive_duplicates, ArrayType(StringType()))

In [8]:
# Group by user_id and session_id, collect event_page as a list, remove consecutive duplicates, and create the route
routes_df = cleaned_data_sql.groupBy("user_id", "session_id") \
    .agg(F.collect_list("event_page").alias("pages")) \
    .withColumn("cleaned_pages", remove_duplicates_udf(F.col("pages"))) \
    .withColumn("route", F.concat_ws("-", "cleaned_pages"))

routes_df.select("user_id", "session_id", "route").show(10, truncate=False)

+-------+----------+-------------------------------------------------------------------------------------------------------------------------+
|user_id|session_id|route                                                                                                                    |
+-------+----------+-------------------------------------------------------------------------------------------------------------------------+
|0      |888       |main                                                                                                                     |
|1      |1026      |main                                                                                                                     |
|10     |762       |main-internet-archive-bonus-internet-main-rabota-main-archive-bonus-news-archive                                         |
|1000   |91        |main-archive-rabota-news-digital-internet-rabota                                                                         |

Spark RDD

In [9]:
# Step 4: Convert to RDD for counting
routes_rdd = routes_df.select("route").rdd.map(lambda row: row.route)

# Count the occurrences of each route
route_counts_rdd = routes_rdd.map(lambda route: (route, 1)) \
    .reduceByKey(lambda a, b: a + b)

# Collect the top 30 most frequent routes
top_routes_rdd = route_counts_rdd.takeOrdered(30, key=lambda x: -x[1])

result_dict = {route: count for route, count in top_routes_rdd}

In [10]:
#Step 5: Print results
with open('result.json', 'w') as json_file:
    json.dump(result_dict, json_file, indent=4)

# Display the result for verification (optional)
print(json.dumps(result_dict, indent=4))

{
    "main": 6054,
    "main-archive": 855,
    "main-rabota": 809,
    "main-internet": 700,
    "main-bonus": 662,
    "main-news": 601,
    "main-tariffs": 519,
    "main-online": 449,
    "main-vklad": 412,
    "main-rabota-archive": 137,
    "main-archive-rabota": 136,
    "main-rabota-bonus": 113,
    "main-bonus-archive": 112,
    "main-news-rabota": 109,
    "main-bonus-rabota": 106,
    "main-internet-rabota": 106,
    "main-internet-archive": 104,
    "main-rabota-news": 104,
    "main-archive-news": 103,
    "main-archive-internet": 101,
    "main-archive-bonus": 100,
    "main-news-archive": 95,
    "main-rabota-internet": 92,
    "main-tariffs-internet": 89,
    "main-internet-bonus": 86,
    "main-internet-news": 81,
    "main-news-bonus": 81,
    "main-rabota-main": 79,
    "main-news-internet": 78,
    "main-archive-tariffs": 77
}
